
# Linear Relaxation of Sigmoid

this notebook is a visualization tests to assess that the linear relaxation of relu is well encoded

In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential

In [2]:
model = Sequential()
model.add(Activation("relu", input_dim=1))

2022-09-20 10:13:32.503104: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import sys

sys.path.append("../..")
from decomon.models import clone

In [4]:
decomon_model = clone(model, method="hybrid")

dict_keys([140616125062608])
dict_keys([140616125062608, 140615590700304])


In [5]:
def relu(x):
    return np.maximum(0.0, x)

In [6]:
lower = -10
upper = 10
x_min = lower * np.ones((1, 1))
x_max = upper * np.ones((1, 1))
box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

output = decomon_model.predict([box])

In [7]:
x, u, w_u, b_u, l, w_l, b_l = output

In [8]:
b_u

array([[5.]], dtype=float32)

In [9]:
decomon_model.inputs

[<KerasTensor: shape=(None, 2, 1) dtype=float32 (created by layer 'input_1')>]

In [10]:
def linear_hull_upper_decomon(lower, upper):
    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)
    # print(box)
    output = decomon_model.predict([box])
    _, _, w_u, b_u = output[:4]
    print(w_u, b_u)

    upper = np.maximum(0.0, w_u) * upper + np.minimum(0.0, w_u) * lower + b_u

    return upper.min(), w_u.min(), b_u.min()


def linear_hull_lower_decomon(lower, upper):

    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

    output = decomon_model.predict([box])
    w_l, b_l = output[-2:]
    lower = np.maximum(0.0, w_l) * lower + np.minimum(0.0, w_l) * upper + b_l
    return lower.min(), w_l.min(), b_l.min()

In [11]:
def func(x_min, x_max):
    n_step = 100
    if x_max < x_min:
        x_max_ = x_max + 0
        x_max = x_min
        x_min = x_max_
    x = np.linspace(x_min, x_max, n_step)
    y = np.linspace(-10, 10, 1000)

    _, w_u, b_u = linear_hull_upper_decomon(x_min, x_max)
    _, w_l, b_l = linear_hull_lower_decomon(x_min, x_max)
    s_x = relu(x)
    s_y = relu(y)
    z_x = w_u * x + b_u
    q_x = w_l * x + b_l

    plt.ylim(-2, 11)

    plt.plot(x, s_x, c="blue")
    plt.plot(x, z_x, c="orange")
    plt.plot(x, q_x, c="green")

    plt.plot(y, s_y, "--", c="k")
    plt.title("Linear Relaxation of ReLU")

In [12]:
interact(
    func,
    x_min=widgets.FloatSlider(value=-10, min=-10, max=10.0, step=0.01, continuous_update=False),
    x_max=widgets.FloatSlider(value=10, min=-10, max=10, step=0.01, continuous_update=False),
)

interactive(children=(FloatSlider(value=-10.0, continuous_update=False, description='x_min', max=10.0, min=-10…

<function __main__.func(x_min, x_max)>